# Data Logger[Averaging Method]
- Need Python 3.7 for firmata4 - use the kernel | Python[conda env:tensorflow_2]
- pymata4 v1.1
- To install 'Firmata Express' on arduino follow [this](https://mryslab.github.io/pymata4/firmata_express).(*Need to install 'Ultrasonic' library*)

In [1]:
from platform import python_version
print(python_version())

3.7.4


In [1]:
# Date : 11.04.2020
# version : 0.3.200411
# Simple averaging method
# > bug: when the position of the sound source change, 
#        the first 5-6 values sometimes remain cloase to the previous levels
# > solutions: > appending data to an array and averaging only suitable ones
#              > use numpy averaging method/ standard diviation method...

import csv
import pyaudio
import numpy as np
import time
import keyboard
from pymata4 import pymata4
from IPython.display import clear_output

direction = 4  # L=0, R=1, F=2, B=3, U=4, D=5

DIGITAL_PIN = 2
TONE_PIN = 3

count = 21     # Since the RATE=44100 and CHUNK=1024, the maximum allowed count = 43/second

CHUNK = 1024
RATE = 44100
CHANNELS = 2
FORMAT = pyaudio.paInt16

p = pyaudio.PyAudio()
stream = p.open(format = FORMAT,
                channels = CHANNELS,
                rate = RATE,
                input = True, 
                frames_per_buffer = CHUNK)

def the_callback(data):
    if data[2]:                                    # If the value of the button press is HIGH
        clear_output(wait = True)
        maxL = []
        maxR = []
        board.play_tone(TONE_PIN, 256, 1000)
        for i in range(count):
            data = np.fromstring(stream.read(CHUNK),dtype = np.int16)
            rawL = data[0::2]
            rawR = data[1::2]            
            #print(np.abs(np.max(rawL)),'-', np.abs(np.max(rawR)))
            maxL.append(np.abs(np.max(rawL)))
            maxR.append(np.abs(np.max(rawR)))
              
        left = np.average(maxL[5:])                # Average by removing the first 5 indicies
        right = np.average(maxR[5:])
        
        row = [left, right, direction]
        with open('data.csv', 'a', newline = '') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
            
        with open('data.csv', "r") as csvfile:     # Print the last recorded value 
            read = csv.reader(csvfile)
            data = list(read)
            idx = len(data)
            print('Last record:','[ ID =',idx,']',data[idx-1])
             
board = pymata4.Pymata4()
board.set_pin_mode_tone(TONE_PIN)
board.set_pin_mode_digital_input(DIGITAL_PIN, callback = the_callback)

while True:      
    if keyboard.is_pressed('Esc'):
        stream.stop_stream()
        stream.close()
        p.terminate()
        board.shutdown()
        break

Last record: [ ID = 1800 ] ['10920.75', '17230.9375', '4']
